**SORA - Stellar Occultation Reduction and Analysis**

## Occultation Object Class

The Occultation Object Class within SORA was created to reduce and analyze stellar occultations, and control all the other Object Classes at SORA package. Here we have some useful tasks that allow converting the times for each observatory in positions in the sky plane ($\xi$, $\eta$), fit an ellipse to the points, obtain the astrometrical position resulting, among others.

The documentation at <font color=blue>add link</font> contains the details about every step.  

This Jupyter-Notebook was designed as a tutorial for how to work with the Occultation Object Class. More information about the other classes, please refer to their specif Jupyter-Notebook. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.

<font size="6">Index</font>

1. [Instantiating the Occultation Object and adding observations](#section_1)

2. [Converting from times to $\xi$ and $\eta$ in the sky plane](#section_2)

3. [Chords display and ellipse fit](#section_3)

4. [Viewing and saving the results](#section_4)

In [1]:
## SORA package
from sora import Occultation, EphemKernel, Star, LightCurve, Observer

## Other main packages
from astropy.time import Time
import astropy.units as u

## Usual packages
import numpy as np
import matplotlib.pylab as pl
import os

<a id="section_1"></a>
<h3 id="section_1">1. Instantiating the Occultation Object and adding observations</h3>

The Occultation Object Class can be instantiated in only one way. For this it is needed a Star Object, an Ephemeris Object (EphemKernel, EphemPlanet or EphemJPL), and the occultation time (within 10 minutes for the correct value).

**The Object Docstring was designed to help the users. Also, each function has its Docstring containing its main purpose and the needed parameters (physical description and formats). Please, do not hesitate to use it.**

In [2]:
Occultation?

Init signature: Occultation(star, body=None, ephem=None, time=None)
Docstring:     
Does the reduction of the occultation
    
Init docstring:
Instantiates the Occultation object.

Parameters:
    star (Star, str): The coordinate of the star in the same reference frame as the ephemeris.
        It must be a Star object or a string with the coordinates of the object to search on
        Vizier (required).
    body* (Body, str): Object that will occult the star. It must be a Body object or its
        name to search in the Small Body Database.
    ephem* (Ephem): object ephemeris. It must be an Ephemeris object or a list .
    time (str, Time): Reference time of the occultation.
        Time does not need to be exact, but needs to be within approximately 50 minutes
        of the occultation closest approach to calculate occultation parameters (required).

* When instantiating with "body" and "ephem", the user may define the Occultation in 3 ways:
    - With "body" and "ephem".
    - Wit

**First let's instanciate the Star and the Ephemeris**

SORA will automatically search for the star information. A warning will raise when any information is missing. In this example, there is no star radius available in Gaia.

In [3]:
ephem_occ = EphemKernel('Chariklo', 2010199, kernels=['input/bsp/Chariklo.bsp','input/bsp/de438_small.bsp'])
star_occ  = Star(coord='18 55 15.65250 -31 31 21.67051') # Occ Chariklo 22-06-2017
time_occ  = '2017-06-22 21:18'

TypeError: __init__() got multiple values for argument 'kernels'

**Now, we can instantiate the Occultation**

In [4]:
occ = Occultation(star_occ, ephem_occ, time_occ)

print(occ)

NameError: name 'star_occ' is not defined

**Note that at the print(Occultation) there are no observations added to this Occultation yet.**

It is needed one Observer and one LightCurve to define an observation.

**Let's instanciate the Observers**

Here we will give 5 locations

In [5]:
out = Observer(name='Outeniqua'  ,lon='+16 49 17.710', lat='-21 17 58.170', height =1416)
ond = Observer(name='Onduruquea' ,lon='+15 59 33.750', lat='-21 36 26.040', height =1220)
tiv = Observer(name='Tivoli'     ,lon='+18 01 01.240', lat='-23 27 40.190', height =1344)
whc = Observer(name='Windhoek'   ,lon='+17 06 31.900', lat='-22 41 55.160', height =1902)
hak = Observer(name='Hakos'      ,lon='+16 21 41.320', lat='-23 14 11.040', height =1843)

**Let's instanciate the LightCurves**

Here we give 6 light curves. Note that 2 are for the same site observed with different telescopes.

In [6]:
out_lc = LightCurve(name='Outeniqua lc',
                    initial_time='2017-06-22 21:20:00.056',
                    end_time ='2017-06-22 21:29:59.963',
                    immersion='2017-06-22 21:21:20.329',immersion_err=0.032,
                    emersion ='2017-06-22 21:21:30.343',emersion_err=0.034)

ond_lc = LightCurve(name='Onduruquea lc',
                    initial_time='2017-06-22 21:11:52.175',
                    end_time ='2017-06-22 21:25:13.389',
                    immersion='2017-06-22 21:21:22.213',immersion_err=0.010,
                    emersion ='2017-06-22 21:21:33.824',emersion_err=0.011)

tiv_lc = LightCurve(name='Tivoli lc',
                    initial_time='2017-06-22 21:16:00.094',
                    end_time ='2017-06-22 21:28:00.018',
                    immersion='2017-06-22 21:21:15.628',immersion_err=0.011,
                    emersion ='2017-06-22 21:21:19.988',emersion_err=0.038)

whc_c14_lc = LightCurve(name='Windhoek C14 lc',
                    initial_time='2017-06-22 21:12:48.250',
                    end_time ='2017-06-22 21:32:47.963',
                    immersion='2017-06-22 21:21:17.609',immersion_err=0.024,
                    emersion ='2017-06-22 21:21:27.564',emersion_err=0.026)

whc_d16_lc = LightCurve(name='Windhoek D16 lc',
                    initial_time='2017-06-22 21:20:01.884',
                    end_time ='2017-06-22 21:22:21.894',
                    immersion='2017-06-22 21:21:17.288',immersion_err=0.028,
                    emersion ='2017-06-22 21:21:27.228',emersion_err=0.034)

hak_lc = LightCurve(name='Hakos lc',
                    initial_time='2017-06-22 21:10:19.461',
                    end_time ='2017-06-22 21:30:19.345')

**To add the observation to the Occultation just use the _Occultation.add_observer()_**

During this step the LightCurve is automatic updated using the Occultation parameters: the Shadow's velocity, the star diameter and the distance to the occulting body. This means that is no need for the user to do the _LightCurve.set_dist()_, _LightCurve.set_vel()_ and _LightCurve.set_star_diam()_.

In [7]:
occ.add_observation(out,out_lc)
occ.add_observation(ond,ond_lc)
occ.add_observation(tiv,tiv_lc)

NameError: name 'occ' is not defined

Note that a warning comes up if the flux drop is not calculated automatically.

**The same goes for two observation at with the same Observer**

In [8]:
occ.add_observation(whc,whc_c14_lc)
occ.add_observation(whc,whc_d16_lc)

NameError: name 'occ' is not defined

**Also, the same goes for negative observations**

In [9]:
occ.add_observation(hak,hak_lc)

NameError: name 'occ' is not defined

**To check which observers were added to the Occultation just use** _Occultation.observations()_

In [10]:
occ.observations()

NameError: name 'occ' is not defined

**If any of them was wrongly added just remove it using** _Occultation.remove_observation()_

In [11]:
occ.remove_observation('Outeniqua')
occ.remove_observation('Hakos lc')

NameError: name 'occ' is not defined

In [12]:
occ.observations()

NameError: name 'occ' is not defined

In [13]:
occ.add_observation(out,out_lc)
occ.add_observation(hak,hak_lc)

NameError: name 'occ' is not defined

<a id="section_2"></a>
<h3 id="section_2">2. Converting from times to $\xi$ and $\eta$ in the sky plane</h3>

This step is done automatically without the user has to ask for it. The user can see its results using the python dictionary Occultation.positions that contains all positions for all observations added.

In [14]:
occ.positions

NameError: name 'occ' is not defined

**Any specif value can be obtained using the keys [Observer][LightCurve]**

In [15]:
occ.positions['Outeniqua']['Outeniqua lc']['immersion']['value']

NameError: name 'occ' is not defined

**The user can save these position**

In [16]:
occ.to_file()

NameError: name 'occ' is not defined

**Even thought it is not needed, the user can use it freely, just import the positionv function within the sora.occultation.**

In [17]:
from sora.occultation import positionv

positionv?

Signature: positionv(star, ephem, observer, time)
Docstring:
Calculates the position and velocity of the occultation shadow relative to the observer.

Parameters:
    star (Star): The coordinate of the star in the same reference frame as the ephemeris.
       It must be a Star object.
    ephem (Ephem): The object ephemeris. It must be an Ephemeris object.
    observer (Observer): The Observer information. It must be an Observer object.
    time (Time): Reference instant to calculate position and velocity.

Return:
    f, g (float): The orthographic projection of the shadow relative to the observer.
        f is in the x-axis (East-West direction; East positive)
        g is in the y-axis (North-South direction; North positive)
File:      ~/miniconda3/lib/python3.8/site-packages/sora/occultation.py
Type:      function


In [18]:
f, g, vel_f, vel_g = positionv(star_occ,ephem_occ,out,'2017-06-22 21:21:22.213')

print('f:  {:+07.3f} km'.format(f))
print('g:  {:+07.3f} km'.format(g))
print('vf: {:+07.3f} km/s'.format(vel_f))
print('vg: {:+07.3f} km/s'.format(vel_g))

NameError: name 'star_occ' is not defined

<a id="section_3"></a>
<h3 id="section_1">3. Chords display and ellipse fit</h3>

The next step is the visualization of the chords in the sky plane and the fitting of an ellipse. The five parameters that describe the ellipse are:

**1.** and **2.** The centre position ($f_0$, $g_0$)

**3.** The apparent equatorial radius, semi-major axix ($a'$)

**4.** The oblatness ($\epsilon' = \frac{a' - b'}{a'}$) 

**5.** The position angle of the pole, semi-minor axix ($P$)

The result of the fit is a ChiSquare Object, and its functions can be found at its specific Jupyter-Notebook.

**Let's look at the chords** 

In [19]:
occ.plot_chords()

##################################################
pl.xlim(+170,-170)
pl.ylim(-170,+170)
pl.show()

NameError: name 'occ' is not defined

**If there are know time shifts, this can be easily solved using** _LightCurve.dt_

In [20]:
out_lc.dt = -0.150
ond_lc.dt = -0.190
tiv_lc.dt = -0.150
whc_c14_lc.dt = -0.375
whc_d16_lc.dt = +0.000

In [21]:
occ.plot_chords()

##################################################
pl.xlim(+170,-170)
pl.ylim(-170,+170)
pl.show()

NameError: name 'occ' is not defined

**Now, let's fit the ellipse**

Here, there is only the manual method, the user should provide the parameters to the fit and also the region for searching each parameters.  

In [22]:
occ.fit_ellipse?

Object `occ.fit_ellipse` not found.


In [23]:
ellipse_chi2  = occ.fit_ellipse(center_f=-15.046, center_g=-2.495, dcenter_f=3, dcenter_g=10, 
                                equatorial_radius=138, dequatorial_radius=3, oblateness=0.093, 
                                doblateness=0.02, position_angle=126, dposition_angle=10 ,loop=100000,
                                dchi_min=10,number_chi=10000)

NameError: name 'occ' is not defined

Note that here we only used 100000 loops, but it is recommended at least 10 millions for a good sample for each parameter.

In [24]:
print(ellipse_chi2)

NameError: name 'ellipse_chi2' is not defined

In [25]:
ellipse_chi2.plot_chi2()

NameError: name 'ellipse_chi2' is not defined

**Now, besides the chords we can plot the fitted ellipse**

In [26]:
from sora.extra import draw_ellipse

draw_ellipse(**ellipse_chi2.get_values())
occ.plot_chords()

##################################################
pl.xlim(+170,-170)
pl.ylim(-170,+170)
pl.show()

NameError: name 'ellipse_chi2' is not defined

**Not just the fitted ellipse, but the user can also plot all the ellipses inside an sigma region, for instance within $3\sigma$**

This step will, possible, plot large number of ellipses, this can spend some CPU time.

In [27]:
draw_ellipse(**ellipse_chi2.get_values())
draw_ellipse(**ellipse_chi2.get_values(sigma=3))

occ.plot_chords()

##################################################
pl.xlim(+170,-170)
pl.ylim(-170,+170)
pl.show()

NameError: name 'ellipse_chi2' is not defined

**The radial velocity issue**

During the LightCurve fitting, the user has to add the velocity for that light curve. In the initial process, only the geocentric velocity was determined, and only after the fit, a radial velocity can be correctly calculated.

Usually, the light curves' features are dominated by the exposure time. However, in the cases that the fresnel diffraction plays a significant role in it, we suggest that the LightCurve fitting procedure should be redone with the correct velocities.

In [28]:
occ.check_velocities()

NameError: name 'occ' is not defined

<a id="section_4"></a>
<h3 id="section_4">4. Viewing and saving the results</h3>

In the end, we can see the results using Python dictionaries. The _Occultation.fitted_params_ will have fitted parameters and their $1\sigma$ uncertainties. The _Occultation.chi2_params_ will have some information about the fit and its quality.



In [29]:
occ.fitted_params

NameError: name 'occ' is not defined

In [30]:
occ.chi2_params

NameError: name 'occ' is not defined

**The astrometrical positions obtained can be seen using** _Occultation.new_astrometric_position()_

In [31]:
occ.new_astrometric_position()

NameError: name 'occ' is not defined

**If the user wants the position at a specific time**

In [32]:
occ.new_astrometric_position('2017-06-22 21:21:00.000')

NameError: name 'occ' is not defined

**Also a 'post-fit' occultation map can be created using** _Occultation.plot_occ_map()_

The fucntion that plots the map contains a large number of _kwargs_. They can be used to complete control the map and its tutorial can be found at SORA documentation <font color=blue>add link</font>     

In [33]:
occ.plot_occ_map(centermap_delta=[-3500,+400],zoom=20,nameimg='output/map_posfit')

NameError: name 'occ' is not defined

<img src='output/map_posfit.png' style='width:600px;height:500px'/>

**All this information can also be seen using** _print(Occultation)_

In [34]:
print(occ)

NameError: name 'occ' is not defined

**And this can be saved to an ASCII file using** _Occultation.to_log_

In [35]:
occ.to_log('output/Test_occ.log')

!ls output/*.log

NameError: name 'occ' is not defined

**This Jupyter-Notebook was designed as a tutorial for how to work with the Occultation Object Class. More information about the other classes, please refer to their specif Jupyter-Notebook. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.**

*The End.*